<a href="https://colab.research.google.com/github/andrew3690/F-A_root_project/blob/Brazil/Data_Ac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing python libraries that will be use to get data from yahoo finance

In [1]:
! pip install yahooquery
! pip install urllib3==1.26
! pip install requests==2.25
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Importing modules that will be used to filter data

In [2]:
from yahooquery import Ticker
from bs4 import BeautifulSoup
import threading
import pandas as pd
import requests
import re

Scrapping Data from Infomoney website, getting a list of stocks from IBOVESPA.

In [3]:
url = requests.get('https://www.infomoney.com.br/cotacoes/empresas-b3/')

soup = BeautifulSoup(url.text, 'html.parser')

ticker_name_list = soup.find_all('td', class_ = 'strong')

ticker = str(ticker_name_list)

tickers = re.findall('>.*</',ticker)

tkrs = []

# strnot = ("11","F","34")
for i in tickers:
  if "11" not in i and "F" not in i:
    tkr = i.strip("</>")
    tkr = tkr + ".SA"
    tkrs.append(tkr)

In [4]:
def fetch_historical_financial(tickers):
    # Yahoo finance requisition getting companies ticker
    sumary = Ticker(tickers)

    # getting companies financial data
    data = sumary.financial_data 

    df = pd.DataFrame(data,  index=[0])

    return df

In [25]:
def fetch_historical_balance(tickers):
    # Balance Ticker lists
    balance = Ticker(tickers)

    # Getting companies balance data
    data = balance.balance_sheet(frequency='a',trailing=False)

    df = pd.DataFrame(data)

    return df

In [6]:
def fetch_historical_finandata(tickers):
    # Balance Ticker lists
    financial = Ticker(tickers)

    # Getting companies balance data
    dt = ['EBIT', 'NetDebt']
    data = financial.get_financial_data(dt,trailing=False)

    df = pd.DataFrame(data)

    return df.T

In [7]:
def fetch_historical_price(tickers):
    pricetkrs = Ticker(tickers)

    dataprice = pricetkrs.price

    df = pd.DataFrame(dataprice)

    return df.T

In [8]:
# Define a function to fetch data for each batch of tickers
def fetch_data_batch(tkrs,FS):
    results = []

    if FS == 1:
        data = fetch_historical_financial(tkrs)
        results.append(data)

    elif FS == 2:
        data = fetch_historical_balance(tkrs)
        results.append(data)

    elif FS == 3:
        data = fetch_historical_finandata(tkrs)
        results.append(data)

    elif FS == 4:
        data = fetch_historical_price(tkrs)
        results.append(data)

    return results

In [13]:
def ElementsList(elements_per_list,tkrs):
    # Divide the ticker list into smaller lists
    smaller_lists = [tkrs[i:i+elements_per_list] for i in range(0, len(tkrs), elements_per_list)]

    # Print the smaller lists
    # for i, smaller_list in enumerate(smaller_lists):
    #     print(f"List {i+1}: {smaller_list}")
    
    return smaller_lists

In [18]:
# Function that processes operations based on the flag of it
# BS: Batch size
# FS: Function Signal
# TL: Ticker List

def Process(BS,TL,FS):

    lists = ElementsList(BS,TL)

    results = fetch_data_batch(lists,FS)

    # Create and start a thread for each ticker batch
    threads = [threading.Thread(target=fetch_data_batch, args=(batch,FS)) for batch in lists]

    # Starting threads
    for thread in threads:
        thread.start()

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    # Consolidate the results
    Financial = pd.concat(results,axis=0, ignore_index=False)

    return Financial

Processing batches of diferent, first 1: historical financial

In [28]:
Finance = Process(19,tkrs,3)

TypeError: sequence item 0: expected str instance, list found

In [20]:
Finance

[  ['ZAMP3.SA', 'N2ET34.SA', 'M1TA34.SA', 'AERI3.SA', 'ICBR3.SA', 'DOTZ3.SA', 'GOLL3.SA', 'ARML3.SA', 'MLAS3.SA', 'CBAV3.SA', 'TTEN3.SA', 'NINJ3.SA', 'ATEA3.SA', 'MODL4.SA', 'MODL3.SA', 'VITT3.SA', 'KRSA3.SA', '.SA', 'CXSE3.SA']  \
 0  Quote not found for ticker symbol: ['ZAMP3.SA'...                                                                                                                                                                                  
 
   ['RIOS3.SA', 'HCAR3.SA', 'GGPS3.SA', 'MATD3.SA', 'ALLD3.SA', 'BLAU3.SA', 'ATMP3.SA', 'ASAI3.SA', 'JSLG3.SA', 'CMIN3.SA', 'ELMD3.SA', 'ORVR3.SA', 'OPCT3.SA', 'WEST3.SA', 'CSED3.SA', 'BMOB3.SA', 'JALL3.SA', 'MBLY3.SA', 'ESPA3.SA']  \
 0  Quote not found for ticker symbol: ['RIOS3.SA'...                                                                                                                                                                                     
 
   ['VAMO3.SA', 'INTB3.SA', 'NGRD3.SA', 'AVLL3.SA', 'RRRP3.SA'

In [ ]:
# # Exporting Excel file Dataframes
# df.to_excel(excel_writer='/content/drive/MyDrive/Data/XLXS/RAW/AtivosBr.xlsx',
#             sheet_name="stocks")

# dfbeta.to_excel(excel_writer='/content/drive/MyDrive/Data/XLXS/RAW/Beta.xlsx'
#             ,sheet_name="key_stats")

# addata.to_excel(excel_writer='/content/drive/MyDrive/Data/XLXS/RAW/Financial.xlsx',
#             sheet_name="financial_data")

# dfprice.to_excel(excel_writer='/content/drive/MyDrive/Data/XLXS/RAW/Price.xlsx',
#             sheet_name="price") 

Local Machine Version

In [ ]:
# df.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/AtivosBR.xlsx',
#            sheet_name="stocks",
#            engine='openpyxl')

# dfbeta.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/Beta.xlsx',
#                 sheet_name="key_stats")

# addata.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/Financial.xlsx',
#             sheet_name="financial_data")

# dfprice.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/Price.xlsx',
#             sheet_name="price")

# balance_sheet.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/Balance.xlsx',
#             sheet_name="balance")